In [ ]:
import requests
import json
import os
from pyspark.sql import SparkSession

# --- 1. Inicialização do Spark ---
# A Spark Session é o ponto de entrada.
spark = SparkSession.builder \
    .appName("IngestaoBronze") \
    .getOrCreate()

print("Sessão Spark iniciada para Ingestão.")

# --- 2. Variáveis de Extração ---
URL_SUICIDIO = "https://ghoapi.azureedge.net/api/SDGSUICIDE?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq 'SEX_BTSX' and Dim2 eq 'AGEGROUP_YEARSALL'"
URL_DEPRESSAO = "https://ghoapi.azureedge.net/api/MH_12?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq 'SEX_BTSX'"

headers = {
    "Content-Type": "application/json"
}

# Caminhos locais temporários para simular a passagem de dados brutos
# Em um ambiente de produção real, estes seriam salvos em um diretório do HDFS/S3 (Camada Bronze).
CAMINHO_BRONZE_SUICIDIO = "dados_bronze_suicidio.json"
CAMINHO_BRONZE_DEPRESSAO = "dados_bronze_depressao.json"

# --- 3. Função de Extração ---
def extrair_api(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Lança um erro para respostas com status 4xx/5xx
        dados = response.json().get("value",[])
        print(f"Dados coletados de {url}. Total de registros: {len(dados)}")
        return dados
        
    except Exception as e:
        print(f"Erro ao chamar a API {url}: {e}")
        return []

# --- 4. Execução da Extração e Salvamento da Camada BRONZE ---
dados_suicidio = extrair_api(URL_SUICIDIO)
dados_depressao = extrair_api(URL_DEPRESSAO)

if not dados_suicidio and not dados_depressao:
    print("Nenhum dado retornado. Encerrando o processo...")
    spark.stop()
    exit()

# Salvando os dados brutos (simulando a Camada Bronze)
with open(CAMINHO_BRONZE_SUICIDIO, 'w') as f:
    json.dump(dados_suicidio, f)

with open(CAMINHO_BRONZE_DEPRESSAO, 'w') as f:
    json.dump(dados_depressao, f)

print(f"Dados BRONZE salvos localmente: {CAMINHO_BRONZE_SUICIDIO} e {CAMINHO_BRONZE_DEPRESSAO}")

# --- 5. Finalização ---
spark.stop()
print("Sessão Spark de Ingestão finalizada.")

Baixando dados de: raw_suicidio...
✅ Arquivo salvo no HDFS: /datalake/bronze/raw_suicidio.json
Baixando dados de: raw_depressao...
✅ Arquivo salvo no HDFS: /datalake/bronze/raw_depressao.json
